# Shortest Paths

* Dijkstra: Brute force
* Dijkstra: Heap-based (much much faster)

In [569]:
import random
import numpy as np

from collections import defaultdict

In [18]:
graph_path = '/home/jacobsuwang/Documents/CS TRAINING/ALGORITHMS/DATA/Dijkstra.txt'
targets = ['7','37','59','82','99','115','133','165','188','197']

### Dijkstra (Base: $O(mn)$)

**Pseudo Code**

$func(G, V, s):$
* $X = {s}$ # put root in 'explored set'
* $A[s] = 0$ # set dist(s,s) to 0
* while $X\neq V$:
    * Find $(v^*,w^*)$ in $\{(v,w)\mid v\in X, w\notin X\}$, s.t. $(v^*,w^*)=\text{argmin}_{(v,w)} A[v]+l_{v,w}$
    * $X := X\cup \{w^*\}$
    * $A[w^*] := A[v] + l_{v^*,w^*}$

In [28]:
def read_graph(path):
    G, V = {}, set()
    with open(path,'r') as source:
        for line in source:
            line = line.split()
            u = line[0]
            V.add(u)
            for tup in line[1:]:
                v, wgt = tup.split(',')
                G[(u,v)] = int(wgt)
    return G,V

In [53]:
G, V = read_graph(graph_path)

def dijkstra_base(G, V, s, targets, verbose=False):
    X = set([s])
    A = {s:0}
    while X!=V:
        min_u,min_v, min_wgt = s,s, np.inf
        for (u,v),wgt in G.iteritems():
            if u in X and v not in X and A[u]+wgt<min_wgt:
                min_u,min_v = u,v
                min_wgt = A[u]+wgt
        X.add(min_v)
        A[min_v] = min_wgt
    if verbose:
        for target in targets:
            print target+':', A[target]

        print
        print 'ANSWER:'
        print targets
        print [A[target] for target in targets]

In [54]:
dijkstra_base(G,V,'1',targets,verbose=1)

7: 2599
37: 2610
59: 2947
82: 2052
99: 2367
115: 2399
133: 2029
165: 2442
188: 2505
197: 3068

ANSWER:
['7', '37', '59', '82', '99', '115', '133', '165', '188', '197']
[2599, 2610, 2947, 2052, 2367, 2399, 2029, 2442, 2505, 3068]


In [55]:
%%timeit

dijkstra_base(G,V,'1',targets)

10 loops, best of 3: 54.9 ms per loop


### Dijkstra (Heap: $O(mlogn)$)

**Pseudo Code**

$func(G, V, s):$
* $X = {s}$
* $A[v]=c,\forall v$, where $c$ is some arbitrary large constant (can use $\infty$) for initial dist(s,v)
* $A[s] = 0$
* initialize heap hp with $hp(s)=0,hp(v)=c,\forall v\in V-\{s\}$
* while $X\neq V$:
    * $u =$ hp.extract_min()
    * $X := X\cup \{u\}$
    * relax($u$) # adjust the $A[v]=\text{min}(A[v],A[u]+l_{u,v}),\forall v\in adj[u]$
    * delete $u$ from heap

In [570]:
def read_graph(path):
    G, V = defaultdict(list), set()
    with open(path,'r') as source:
        for line in source:
            line = line.split()
            u = line[0]
            V.add(u)
            for tup in line[1:]:
                v, wgt = tup.split(',')
                G[u].append((v,int(wgt)))
    return G,V      

class Node:
    
    def __init__(self, name, val):
        self.name = name
        self.val = val
        
class NodeHeap:
    
    def __init__(self):
        self.__heap = []
        self.__node2idx = {}
        self.__len = 0
        
    # Private Methods
    
    def _is_empty(self):
        return True if self.__len==0 else False
        
    def _swap(self, u, v):
        u_idx,v_idx = self.__node2idx[u],self.__node2idx[v]
        self.__heap[u_idx],self.__heap[v_idx] = \
        self.__heap[v_idx],self.__heap[u_idx]
        self.__node2idx[u],self.__node2idx[v] = v_idx,u_idx
        
    def _get_parent(self, u):
        try: return self.__heap[(self.__node2idx[u]-1)//2].name
        except: return False
    def _get_leftchild(self, u):
        try: return self.__heap[2*self.__node2idx[u]+1].name
        except: return False
    def _get_rightchild(self, u):
        try: return self.__heap[2*self.__node2idx[u]+2].name
        except: return False
    
    def _remove_last(self):
        if not self._is_empty():
            last_node = self.__heap.pop()
            del self.__node2idx[last_node.name]
            self.__len -= 1
    
    # Public Methods
    
    def heap(self):
        return self.__heap
    def print_heap(self):
        return [(node.name,node.val) for node in self.__heap]
    def node2idx(self):
        return self.__node2idx
    def length(self):
        return self.__len
    def get_name(self, u_idx):
        if not self._is_empty(): return self.__heap[self.__len-1].name
        return False
    def get_index(self, u):
        if not self._is_empty(): return self.__node2idx[u]
        return False
    def get_value(self, u):
        if not self._is_empty(): return self.__heap[self.get_index(u)].val
        return False
        
    def insert(self, u, val):
        self.__heap.append(Node(u, val))
        self.__node2idx[u] = self.__len
        self.__len += 1
        while True:
            if self.get_index(u)==0: break
            parent_u = self._get_parent(u)
            if self.get_value(u)<self.get_value(parent_u):
                self._swap(u,parent_u)
            else: break
    
    def delete(self, u):
        v = self.get_name(self.__len-1)
        self._swap(u, v)
        self._remove_last()
        u = v
        while True:
            lc_u,rc_u = self._get_leftchild(u),self._get_rightchild(u)
            if lc_u and rc_u:
                v = lc_u if self.get_value(lc_u)<self.get_value(rc_u) else rc_u
            elif lc_u and not rc_u:
                v = lc_u
            else:
                break
            if self.get_value(u)<self.get_value(v):
                break
            self._swap(u, v)
    
    def extract_min(self):
        if not self._is_empty():
            return self.__heap[0].name
        return False

In [571]:
def test():
    names = ['a','b','c','d','e','f','g']
    vals = [4,7,14,12,9,17,18]  
    scramble = zip(names,vals)
    random.shuffle(scramble)
    print 'INPUT:', scramble
    print
    print 'INSERT TEST:'
    nhp = NodeHeap()
    for name,val in scramble:
        nhp.insert(name,val)
        print 'inserted', name+', got', nhp.print_heap(), '(min='+nhp.extract_min()+')'
    print
    print 'DELETE TEST:'
    for name,val in scramble:
        nhp.delete(name)
        print 'deleted', name+', got', nhp.print_heap(), 
        min_name = nhp.extract_min()
        print '(min='+min_name+')' if min_name else '()'
        
test()

INPUT: [('e', 9), ('c', 14), ('b', 7), ('f', 17), ('d', 12), ('a', 4), ('g', 18)]

INSERT TEST:
inserted e, got [('e', 9)] (min=e)
inserted c, got [('e', 9), ('c', 14)] (min=e)
inserted b, got [('b', 7), ('c', 14), ('e', 9)] (min=b)
inserted f, got [('b', 7), ('c', 14), ('e', 9), ('f', 17)] (min=b)
inserted d, got [('b', 7), ('d', 12), ('e', 9), ('f', 17), ('c', 14)] (min=b)
inserted a, got [('a', 4), ('d', 12), ('b', 7), ('f', 17), ('c', 14), ('e', 9)] (min=a)
inserted g, got [('a', 4), ('d', 12), ('b', 7), ('f', 17), ('c', 14), ('e', 9), ('g', 18)] (min=a)

DELETE TEST:
deleted e, got [('a', 4), ('d', 12), ('b', 7), ('f', 17), ('c', 14), ('g', 18)] (min=a)
deleted c, got [('a', 4), ('d', 12), ('b', 7), ('f', 17), ('g', 18)] (min=a)
deleted b, got [('a', 4), ('d', 12), ('g', 18), ('f', 17)] (min=a)
deleted f, got [('a', 4), ('d', 12), ('g', 18)] (min=a)
deleted d, got [('a', 4), ('g', 18)] (min=a)
deleted a, got [('g', 18)] (min=g)
deleted g, got [] ()


In [572]:
def init_heap(V, s):
    hp = NodeHeap()
    hp.insert(s, 0)
    for v in V-set([s]):
        hp.insert(v,1000000)
    return hp

def relax(G, X, A, hp, u):
    for v,wgt in G[u]:
        if v not in X:
            hp.delete(v)
            A[v] = min(A[v],A[u]+wgt)
            hp.insert(v,A[v])

def dijkstra(G, V, s, targets, verbose=False):
    X = set([s])
    A = {v:1000000 for v in V}
    A[s] = 0
    hp = init_heap(V, s)
    while X!=V:
        u = hp.extract_min()
        X.add(u)
        relax(G, X, A, hp, u)
        hp.delete(u)
    if verbose:
        for target in targets:
            print target+':', A[target]

        print
        print 'ANSWER:'
        print targets
        print [A[target] for target in targets]

In [575]:
G,V = read_graph(graph_path)

dijkstra(G,V,'1',targets,verbose=1)

7: 2599
37: 2610
59: 2947
82: 2052
99: 2367
115: 2399
133: 2029
165: 2442
188: 2505
197: 3068

ANSWER:
['7', '37', '59', '82', '99', '115', '133', '165', '188', '197']
[2599, 2610, 2947, 2052, 2367, 2399, 2029, 2442, 2505, 3068]


In [574]:
%%timeit

dijkstra(G,V,'1',targets)

100 loops, best of 3: 13.6 ms per loop
